#Class Definitions

In [237]:
import iso8601

class Dossier(object):
    """
        The Dossier object is designed to contain all metadata for a single dossier.
        It can be initialized by passing the EnergyID metadata json to the constructor,
        or by calling the parse_EID_json method.
        Parsing the JSON is optional and put in a seperate method so there is room
        to add other metadata formats.
    """
    def __init__(self, EID_json=None):
        """
            Object constructor. If the EnergyID JSON is passed, all metadata and
            datapoints are parsed and added. It is left optional though so other methods
            of initializing can be added in the future.
            
            Parameters
            ----------
            EID_json: JSON string from EnergyID containing all metadata for a single dossier
        """
        self.points = []
        if EID_json is not None:
            self._parse_EID_json(EID_json)
    
    def _parse_EID_json(self,mjson):
        """
            Method that parses the EnergyID JSON
            
            Parameters
            ----------
            mjson: JSON string from EnergyID containing all metadata for a single dossier
        """
        
        self.id = mjson['id']
        self.version = mjson['version']
        self.created = iso8601.parse_date(mjson['created']) #parse to datetime
        self.active = mjson['status'] == 'active' #parse to bool
        
        #We save the context and weatherstation metadata as raw JSON for now.
        self._context = mjson['context']
        self.weatherStations = mjson['weatherStations']
        
        #Data Points are objects. We create a list of objects here.
        for pjson in mjson['points']:
            self.points.append(Point(dossier = self, EID_json = pjson))
            
    def get_property(self,tag):
        """
            Method that parses context metadata
            
            Parameters
            ----------
            tag: String, tag of the property to be parsed
            
            Returns
            -------
            Parsed value for the given tag.
            None if the tag was not found.
        """
        try:
            res = self._context[tag]
        except KeyError:
            res = None
            
        return res
        
    def find_dimension(self, name, value):
        """
            Search all points for a dimension
            
            Parameters
            ----------
            name: String, name of the dimension
            value: String, value of the dimension
            
            Returns
            -------
            Dimension object
            None if no object was found
        """
        for point in self.points:
            dimension = point.find_dimension(name, value)
            if dimension is not None:
                return dimension
        return None
    
    def find_metric(self, name):
        """
            Search all points for a metric
            
            Parameters
            ----------
            name: String, name of the metric
            
            Returns
            -------
            Metric object
            None if no object was found
        """
        for point in self.points:
            metric = point.find_metric(name)
            if metric is not None:
                return metric
        return None

In [238]:
#define a small method to substitute 'none' by None
def parse_none(text):
    if text == 'none' or text == '':
        return None
    else:
        return text

In [255]:
class Point(object):
    """
        The Point object countains all metadata for a Data Point,
        and all coupled dimensions and metrics.
    """
    def __init__(self, dossier, EID_json=None):
        """
            Construction method. EnergyID JSON is optional so that
            a data point can be constructed by via another method in the future.
            
            Parameters
            ----------
            dossier: Dossier object, parent to this Data Point
            EID_json: EnergyID JSON for this data Point (optional)
        """
        self.dossier = dossier
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
            
    def _parse_EID_json(self,mjson):
        """
            Parse the EnergyID JSON into metadata, metrics and dimensions
            
            Parameters
            ----------
            mjson: JSON String, EnergyID JSON for a single point
        """
        self.id = mjson['id']
        self.active = mjson['status'] == 'active' #parse to bool
        
        #save al other properties as raw json and parse when requested
        self._raw_json = mjson
        
        #create list of coupled metrics and dimensions
        self.metric_couplings = [Metric_Coupling(point = self, EID_json = cjson) for cjson in mjson['metrics']]
        
    def get_propery(self,tag):
        """
            Method that parses context metadata
            
            Parameters
            ----------
            tag: String, tag of the property to be parsed
            
            Returns
            -------
            Parsed value for the given tag.
            None if the tag was not found.
        """
        try:
            res = self._raw_json[tag]
        except KeyError:
            res = None
            
        return res
    
    def find_dimension(self, name, value):
        """
            Seach for a given dimension in all coupled dimensions
            
            Parameters
            ----------
            name: String, name of the dimension
            value: String, value of the dimension
            
            Returns
            -------
            Dimension object
            None if no object found
        """
        for metric_coupling in self.metric_couplings:
            dimension = metric_coupling.find_dimension(name, value)
            if dimension is not None:
                return dimension
        return None
    
    def find_metric(self, name):
        """
            Seach for a metric in all coupled metrics
            
            Parameters
            ----------
            name: String, name of the metric
            
            Returns
            -------
            Metric object
            None if no object found
        """
        for metric_coupling in self.metric_couplings:
            if metric_coupling.metric.name == name:
                return metric_coupling.metric
        return None

In [256]:
class Metric_Coupling(object):
    """
        Intermediate Object designed to couple a Metric and Dimensions to a single Point.
    """
    def __init__(self, point, EID_json = None):
        """
            Constructor
            
            Parameters
            ----------
            point: Point object, parent
            EID_json: EnergyID JSON, optional
        """
        self.point = point
        
        if EID_json is not None:
            self._parse_EID_json(EID_json)
    
    def _parse_EID_json(self, mjson):
        """
            Parse EnergyID JSON to dimensions and metric
            
            Parameters
            ----------
            mjson: EnergyID JSON
        """
        self.coefficient = mjson['coefficient']
        
        #create list of dimensions
        self.dimensions = [self.find_create_dimension(name = djson['name'], value = djson['value']) for djson in mjson['dimensions']]
        
        #find or create the coupled metric
        self.metric =  self.find_create_metric(name = mjson['name'])
        
    def find_create_dimension(self, name, value):
        """
            Find or create a dimension.
            Look if the dimension exists in this dossier, if not: create it.
            
            Parameters
            ----------
            name: String, name of the dimension
            value: String, value of the dimension
            
            Returns
            -------
            Dimension object
        """
        dimension = self.point.dossier.find_dimension(name = name, value = value)
        
        if dimension is None:
            dimension = Dimension(name = name, value = value)
          
        #add the coupling to the dimension, for reverse lookup
        dimension.metric_couplings.append(self)
            
        return dimension
    
    def find_create_metric(self, name):
        """
            Search for a metric inside the dossier. If it doesn't exist, create a new one.
            
            Parameters
            ----------
            name: String, name of the metric
            
            Returns
            -------
            Metric object
        """
        metric = self.point.dossier.find_metric(name)
        
        if metric is None:
            metric = Metric(name)
           
        #add the coupling to the metric, for reverse lookup
        metric.metric_couplings.append(self)
            
        return metric
    
    def find_dimension(self, name, value):
        """
            Search for a dimension
            
            Parameters
            ----------
            name: String
            value: String
            
            Returns
            -------
            Dimension object
            None if no object found
        """
        for dimension in self.dimensions:
            if dimension.name == name and dimension.value == value:
                return dimension
        return None

In [241]:
class Dimension(object):
    """
        Object contains info for a dimension,
        and all metrics and points that use this dimension
    """
    def __init__(self, name, value):
        """
            Constructor
            
            Parameters
            ----------
            name: String
            value: String
        """
        self.name = name
        self.value = value
        self.metric_couplings = []

In [257]:
class Metric(object):
    """
        Object contains info for a metric,
        and all points and dimensions that are coupled to this metric
    """
    def __init__(self, name):
        """
            Constructor
            
            Parameters
            ----------
            name: String
            
        """
        self.name = name
        self.metric_couplings = []

#Parsing

In [258]:
import json

In [259]:
with open('metadata_eid.json') as data_file:    
    mjson = json.load(data_file)

In [260]:
d = Dossier(EID_json=mjson)

In [261]:
print d.id, d.version, d.created, d.active, d.context

14092349 1 2010-12-08 16:17:09+00:00 True {u'category': u'dwelling', u'principalResidence': True, u'auxiliaryHeatingOn': u'none', u'householdSize': 5, u'cookingOn': u'gas', u'city': u'Vremde', u'dwellingType': u'semiDetached', u'country': u'BE', u'renovationYear': u'<2005', u'floorSurface': 180, u'constructionYear': u'<1985', u'installations': [], u'hotWaterOn': u'gas', u'heatingOn': u'gas', u'energyPerformance': 346, u'postalCode': u'2531', u'occupier': u'owner', u'energyEfficiency': u'traditional'}


In [262]:
d.get_context('category'), d.get_context('dwellingType')

(u'dwelling', u'semiDetached')

In [263]:
for w in d.weatherStations:
    print w

{u'name': u'BRU'}


In [267]:
for p in d.points:
    print p.id
    print p.get_propery('displayName')
    print p.metric_couplings
    print '\n'

496d64a3-1177-4370-91c8-39d0e5cce157
Elektriciteitsmeter (2) (dag)


a42489f0-879e-4570-87f5-4b86d519d508
Zonneboiler (2)


b049d374-0326-4637-a45d-5f43330003cd
Aardgasmeter


5845893a-42d7-40c8-a746-6044df39195b
Zonneboiler


e4e7d03d-e093-4952-befd-684e5b08cc06
Elektriciteitsmeter nieuw


930fabdb-8c23-4f3e-a1b6-a00b0d2d804b
Kilometerteller Audi


0fd5e9c5-664f-4ca6-8e26-a7711ac5221b
Elektriciteitsmeter (2) (nacht)


edf7b513-d2c6-4253-9322-a9d4df5dcbc4
Kilometerteller Peugeot


515ae7fb-dca6-41e0-9217-c3a1043ce91c
PV panelen
[<__main__.Metric_Coupling object at 0x103951310>, <__main__.Metric_Coupling object at 0x103951410>]


10d598fb-d307-46fa-9b12-f93972af1851
Watermeter


ea7bd780-46fd-4afa-8b0f-faebc0ad477f
Kilometerteller Skoda




In [268]:
for p in d.points:
    for mc in p.metric_couplings:
        print mc.coefficient
        print mc.metric, mc.metric.name
        for dim in mc.dimensions:
            print dim.name, dim.value
    print '\n'

1
<__main__.Metric object at 0x10389b8d0> ea:powerConsumption
ea:contentType electricity
ea:source grid
ea:system main


1
<__main__.Metric object at 0x10389b190> ea:heatProduction
ea:contentType heat (renewable)
ea:generationSystem solarBoiler
ea:system Zonneboiler


1
<__main__.Metric object at 0x1020a8350> ea:gasConsumption
ea:contentType natural gas
ea:source grid
ea:system main


1
<__main__.Metric object at 0x10389b190> ea:heatProduction
ea:contentType heat (renewable)
ea:generationSystem solarBoiler
ea:system Zonneboiler


1
<__main__.Metric object at 0x10389b8d0> ea:powerConsumption
ea:contentType electricity
ea:source grid
ea:system main


1
<__main__.Metric object at 0x10209f850> ea:distanceTravelled
ea:contentType distance
ea:vehicle car
ea:system main


1
<__main__.Metric object at 0x10389b8d0> ea:powerConsumption
ea:contentType electricity
ea:source grid
ea:system main


1
<__main__.Metric object at 0x10209f850> ea:distanceTravelled
ea:contentType distance
ea:vehicle car
e